In [ ]:
%load_ext autoreload
%autoreload 2
    
from bmp_config import path_data,envcode2env
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from bmp_behav_proc import *

import numpy as np
from pingouin import ttest
from bmp_base import radius_cursor
import bmp_behav_proc

In [ ]:
from datetime import datetime

fnf = pjoin(path_data,'df_all_multi_tsz__.pkl.zip')
print(fnf)
print( str(datetime.fromtimestamp(os.stat(fnf).st_mtime)))
df_all_multi_tsz = pd.read_pickle(fnf)
print(df_all_multi_tsz['trial_group_col_calc'].unique())
print(df_all_multi_tsz['retention_factor_s'].unique())
# df = df_all_multi_tsz.query('trial_shift_size == 1 and trial_group_col_calc == "trialwe" '
#                             ' and retention_factor_s == "1.000"').copy()
df_all_multi_tsz_orig = df_all_multi_tsz

In [ ]:
df = df_all_multi_tsz.query('trial_shift_size == 1 and trial_group_col_calc == "trialwe" '
                           ' and retention_factor_s == "0.924"').copy().sort_values(['subject','trials'])
df_orig = df
# define dfall here
df,dfall,ES_thr,envv,pert = addBehavCols2(df);


In [ ]:
# prepare for generalization analysis
import bmp_behav_proc
stds = bmp_behav_proc._calcStds(df)
stds = stds.to_frame().reset_index().rename(columns={'error_deg':'error_deg_initstd'}) 

print( df_all_multi_tsz.trial_group_col_calc.unique() )

df_all_multi_tsz_whtr = df_all_multi_tsz.merge(stds, on='subject')
df_all_multi_tsz_whtr_ = df_all_multi_tsz_whtr.\
    query('trial_group_col_calc in ["trialwe", "trialwtgt_we"]').copy()

df_all_multi_tsz_whtr_['env'] = df_all_multi_tsz_whtr_['environment'].apply(lambda x: envcode2env[x])

df_all_multi_tsz_whtr_['error_deg'] = df_all_multi_tsz_whtr_['error'] / np.pi * 180 
df_all_multi_tsz_whtr_['error_initstd'] = df_all_multi_tsz_whtr_.error_deg_initstd /  180 * np.pi 

print('trial_group_col_calc uniue = ', df_all_multi_tsz['trial_group_col_calc'].unique() )


checkErrBounds(df,['error','prev_error','error_deg'])
checkErrBounds(df_all_multi_tsz_whtr_,['error','prev_error','error_deg'])

shiftszs = df_all_multi_tsz_whtr_['trial_shift_size'].unique()
print(shiftszs)

_cols = [col for col in df_all_multi_tsz.columns if col.find('like') >= 0]
print(_cols)

# truncate multi-ver ES
# remove trials with error > std_mult * std of error
dfall_notclean = df_all_multi_tsz_whtr_.query('err_sens.abs() <= @ES_thr')

dfall_mshsz = truncateDf(dfall_notclean, 'err_sens', q=0.0, infnan_handling='discard', 
       cols_uniqify = ['subject','environment','trial_shift_size',
                       'trial_group_col_calc','retention_factor_s'])


In [ ]:
dfc_multi_tsz = dfall_mshsz.query('retention_factor_s == "0.924"')

In [ ]:
print('min,max ES {:.3f}, {:.3f}'.format( dfc_multi_tsz.err_sens.min(), dfc_multi_tsz.err_sens.max() ) )
assert dfc_multi_tsz['retention_factor_s'].nunique() == 1

In [ ]:
df_ = dfc_multi_tsz.query('trial_shift_size > 1 and trial_group_col_calc == "trialwe"')
neq = (df_['dist_rad_from_prevtgt_shiftrespect'] != df_['dist_rad_from_prevtgt2']).sum()
eq = (df_['dist_rad_from_prevtgt_shiftrespect'] == df_['dist_rad_from_prevtgt2']).sum()
print( neq/len(df_), eq/len(df_))

In [ ]:
#distcol = 'dist_rad_from_prevtgt2'
distcol = 'dist_rad_from_prevtgt_shiftrespect'

drs = dfc_multi_tsz[distcol].unique();
drs_f = np.array( list(map( float, drs) ) )
drs = list(sorted(drs,key = lambda x: float(x)))


#dfc_s = dfc[dfc['trial_group_col_calc'] == 'trialwtgt_we']
# TODO: should I restrict to one env?
dfc_s = dfc_multi_tsz.query('trial_group_col_calc == "trialwe"')
grp = dfc_s.groupby(['trial_shift_size', distcol])
dts = dfc_multi_tsz['trial_shift_size'].unique()
nt = len(dts)

print(drs, dts)

assert nt == 4, nt
assert len(drs) == 4, len(drs)

In [ ]:
drpairs = [(drs[i],drs[i+1] ) for i in range(3)]
drpairs += [(drs[0],drs[i] ) for i in range(2,4)]
drpairs += [(drs[1],drs[3] )]
print(drpairs)

dtpairs = [(dts[i],dts[i+1] ) for i in range(3)]
dtpairs += [(dts[0],dts[i] ) for i in range(2,4)]
dtpairs += [(dts[1],dts[3] )]
dtpairs

In [ ]:
from figure.plots import plotPolys_genplot as plotPolys
from figure.plots import getPvals_genplot as getPvals

In [ ]:
from figure.mystatann import plotSigAll, plotSig0All
from figure import env_order,palette_stabrand,env2color
# not pooled
#%debug
text_shift = (-60, 20)
annot_loc = (0, 1.03)

hue = 'env'
mcorr = 'holm'
#env2color = {'stable':'lightblue', 'random':'orange'}
aspect=1.3
#rng = [2]
#rng = [2,3,4,5,6,7]
rng = [2]
ylim = [-2,3]
ylim_plot = [-0.7,2.5]
meanfit=1
#qs_spatial = f'trial_group_col_calc == "trialwe" and err_sens >= {ylim[0]} and err_sens <= {ylim[1]} and trial_shift_size == 1'
qs_spatial = f'trial_group_col_calc == "trialwe" and trial_shift_size == 1'
qs_temporal = f'trial_group_col_calc == "trialwe" and {distcol} == "0.00"'
qs_temporal_naive = f'trial_group_col_calc == "trialwe"'  # use all trials
#bnqs = [  ('grid_line_violin_spatial','Spatial distance [deg]',drs,drpairs,'dist_rad_from_prevtgt2', f'trial_group_col_calc == "trialwe" and err_sens >= {ylim[0]} and err_sens <= {ylim[1]} and trial_shift_size == 1'),
#        ( 'grid_line_violin_temporal','Temporal distance [number of trials]',dts,dtpairs,'trial_shift_size','trial_group_col_calc == "trialwe" and dist_rad_from_prevtgt2 == "0.00"')]
bnqs = {'grid_line_violin_spatial': ('Spatial distance [deg]',drs,drpairs,
            distcol, qs_spatial),
        'grid_line_violin_temporal':('Temporal distance [number of trials]',dts,dtpairs,
            'trial_shift_size',qs_temporal),
       'grid_line_violin_temporal_naive':('Temporal distance [number of trials]',dts,dtpairs,
            'trial_shift_size',qs_temporal_naive)}
panel_letters = ['A','B','B','C','D','D']
paneli  = 0

dfr = []
ttrs0s = []
for env_type in ['stable','random']:
    color = env2color[env_type]
#    for bn,xlab,order,pairs,fitcol,qs in bnqs:
    for bn,(xlab,order,pairs,fitcol,qs) in bnqs.items():
        #qs0 = f'trial_group_col_calc == "trialwe" and err_sens >= {ylim[0]} and err_sens <= {ylim[1]}'
        #qs = qs + ' and trial_shift_size == 1'
        if env_type in ['stable', 'random']:
            qs += ' and env == @env_type'
            hue = None
            
        dftmp0 = dfc_multi_tsz.query(qs).groupby(['subject','environment',fitcol])
        dftmp0 = dftmp0.mean(numeric_only = True).reset_index()                
        dfr += [dftmp0]

        #dftmp0 = dfc_multi_tsz.query(qs)
        #fitcol = 'dist_rad_from_prevtgt2'
        fg = sns.catplot(kind='violin', data=dftmp0, y='err_sens', 
                         hue=hue, x=fitcol, order = order,
                          color=color, aspect=aspect, cut=True)
        #dftmp = dfc_multi_tsz.query(qs).copy()
        dftmp = dftmp0.copy()
        #pvalues, formatted_pvalues = getPvals(dftmp,fitcol, pairs)

        ax = fg.axes.flatten()[0]
        ttrs,ttrs_sig = plotSigAll(ax, 1.3, 0.2, 0.05, df=dftmp, 
                   coln = 'err_sens', colpair = fitcol,
                  verbose = 0, multi_comp_corr_method=mcorr)
        ttrs0,ttrs0_sig = plotSig0All(ax, 2.5, txt=None, df=dftmp, 
                   coln = 'err_sens', colpair = fitcol,
                  multi_comp_corr_method=mcorr, graded_signif = True, show_ns = True)
        ttrs0['bn'] = bn
        ttrs0['env'] = env_type
        ttrs0['fitcol'] = fitcol
        ttrs0s += [ttrs0]

        if bn.find('_spatial') >= 0:
            pref = 'Spatial generalization in \n'
        else:
            pref = 'Temporal generalization in \n'
        for ax in fg.axes.flatten():
            ax.axhline(0,ls=':',c='r')
            ax.set_ylabel('Error sensitivity', fontdict={'fontsize':12})
            ax.set_xlabel(xlab, fontdict={'fontsize':12} )
            if bn == 'grid_line_violin_spatial':
                ax.set_xticklabels(['same target', 45, 90, 135])            
            ax.set_title(pref + f'{env_type} environment', y = 1,
                        fontdict={'fontsize':15, 'fontweight':'bold'})

        ax.annotate(panel_letters[paneli], xy=annot_loc, xytext=text_shift, 
        fontsize=19, fontweight='bold', va='top', ha='left',
            xycoords='axes fraction', textcoords='offset points')

        ax.set_ylim(ylim_plot)

        plotPolys(ax,dftmp,fitcol, degs=rng, mean=meanfit)
        plt.tight_layout()
        #bn = 'grid_line_violin_spatial'
        plt.savefig(pjoin(path_fig,'behav',f'nopool_{env_type}_{bn}.svg'))
        plt.savefig(pjoin(path_fig,'behav',f'nopool_{env_type}_{bn}.pdf'))
        paneli += 1

    #     break
    # break
ttrs0s = pd.concat(ttrs0s, ignore_index=True)

In [ ]:
ttrs0s[['bn', 'env', 'T','pval','alt','ttstr','fitcol','varval','mc_corr_method',
        'trial_shift_size','dist_rad_from_prevtgt_shiftrespect']]

In [ ]:
# make a 2x2 file
from figure.imgfilemanip import  stackSVGandShowJupy, svg2png
svg_fnames = []
#use_all_trials_for_temporal = True # True gives naive, flase give first version 
use_all_trials_for_temporal = False  # what will go to main 4x4 pic
bnsuff2fnsuff={'_naive':'_naive_temporal', '':'' }  # bn suffix to file name suffix

if use_all_trials_for_temporal:
    suff_bn = '_naive'; suff_bn_opposite = '' #; suff2= '_naive_temporal'  
else:
    suff_bn = ''; suff_bn_opposite = '_naive'  #; suff2= ''
suff2 = bnsuff2fnsuff[suff_bn]
for env_type in ['stable','random']:
    for bn in ['grid_line_violin_spatial','grid_line_violin_temporal' + suff_bn]:
        fnf = pjoin(path_fig,'behav',f'nopool_{env_type}_{bn}.svg')
        svg_fnames += [fnf]

show = 0
fnfout1 = pjoin(path_fig,'behav',f'nopool_stable{suff2}.svg')
stackSVGandShowJupy(svg_fnames[:2], 'horizontal', fnfout1, show=show) # stable horizontally
fnfout2 = pjoin(path_fig,'behav',f'nopool_random{suff2}.svg')
stackSVGandShowJupy(svg_fnames[2:], 'horizontal', fnfout2, show=show) # random horizontally

fnfout = pjoin(path_fig,'behav',f'generalization_2x2_nopool{suff2}.svg')
stackSVGandShowJupy([fnfout1,fnfout2], 'vertical', fnfout, show=show)

svg2png(fnfout, dpi=300)

In [ ]:
svg_fnames = []
for env_type in ['stable','random']:
    for bn in ['grid_line_violin_temporal' + suff_bn_opposite]:
        fnf = pjoin(path_fig,'behav',f'nopool_{env_type}_{bn}.svg')
        svg_fnames += [fnf]

suff2 = bnsuff2fnsuff[suff_bn_opposite]

show = 0
fnfout1 = pjoin(path_fig,'behav',f'nopool_temporal{suff2}.svg')
stackSVGandShowJupy(svg_fnames[:2], 'vertical', fnfout1, show=show)

svg2png(fnfout1, dpi=300)

# Stats

In [ ]:
# compute mean pearson corr between temporal/spatial distance and err sens
dfr = []
method='spearman'
for bn,(xlab,order,pairs,fitcol,qs) in bnqs.items():
    dftmp0 = dfc_multi_tsz.query(qs).groupby(['subject','env',fitcol])
    dftmp0 = dftmp0.mean(numeric_only = True).reset_index()
    #dftmp0['thr'] = "_"
    dftmp0[fitcol] = dftmp0[fitcol].astype(float)
    corrme,corrme_sep = corrMean(dftmp0, coltocorr=fitcol, 
        stagecol='env' , coln='err_sens', method=method)
    corrme_sep['antype'] = bn
    dfr += [corrme_sep.reset_index()]
dfr = pd.concat(dfr, ignore_index=True)

In [ ]:
ttrss = []
for antype in bnqs.keys():
    for env in dfr.env.unique():
        ttrs = compare0(dfr.query('antype == @antype and env == @env'), 'r', ['less'], 'r')
        ttrs['antype'] = antype
        ttrs['env'] = env
        ttrss += [ttrs]
ttrs = pd.concat(ttrss, ignore_index=True)
print(len(ttrs.query('pval <= 0.05')))
assert len(ttrs.query('pval <= 0.05')) == 4

In [ ]:
# slope sign
temporal_antype_to_use = 'grid_line_violin_temporal_naive'
s0 = ''
for i,row in ttrs.iterrows():
    antype = row['antype']
    if (antype.find('temporal') >= 0) and (antype != temporal_antype_to_use):
        continue
    ant = antype.split('_')[-1]        
    #row['alt']
    ttsshrt = row['ttstr'][-3:]
    s = 'for {} environment the the mean {} correlation of ES versus {} distance is {:.2f}+/-{:.2f} {}, p-value = {:.2e}'.format(
        row['env'], method, ant, row['r_mean'], row['r_std'], ttsshrt, row['pval']
    )
    #print(s)
    s0 += s[0].upper() + s[1:] + '. \n'
print(s0)

In [ ]:
ttrss = []
for antype in bnqs.keys():    
    ttrssig,ttrs = comparePairs(dfr.query('antype == @antype'), 'r', 'env', 
            alt = ['greater','less','two-sided'], 
                        paired=True, pooled=0 )
    ttrs['antype'] = antype
    #ttrs['env'] = env
    ttrss += [ttrs]
ttrs = pd.concat(ttrss, ignore_index=True)
#assert len(ttrs.query('pval <= 0.05')) == 4

In [ ]:
display(ttrs)
ttrssig = ttrs.query('pval <= 0.05 and alternative == "less"')
print('sig only')
display(ttrssig)

In [ ]:
ttrs.query('alternative == "two-sided" and antype == "grid_line_violin_temporal"')

In [ ]:
#ttrs_sig
if len(ttrssig):
    print('!! SOME SIG !!!')
    pv = ttrssig.iloc[0]['pval']
else:
    pv = ttrs.query('alternative == "two-sided" and antype == "grid_line_violin_temporal"').iloc[0]['pval']
s_ = '{:.2e}'.format( pv )
pvtemp = pv

pv = ttrs.query('alternative == "two-sided" and antype == "grid_line_violin_spatial"').iloc[0]['pval']
pvspat = pv
s_2 = '{:.2e}'.format( pv )
if len(ttrssig):
    s = f'Temporal slope in random is stronger negative than in stable, p-value={s_}. '
else:
    s = f'Temporal slopes in random and stable are not different, p-value={s_}. '
s += f'Spatial slopes in random and stable are not different, p-value={s_2}. '
#print(s)


print(s0,'\n',s)

In [ ]:
print(f'{pvspat:.2e}, {pvtemp:.2e}')